#Setup

In [ ]:
!nvidia-smi

Fri Sep 16 00:09:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q diffusers
!pip install -q transformers scipy ftfy
!pip install -q "ipywidgets>=7,<8"
!pip install -q gradio

     |████████████████████████████████| 153 kB 7.0 MB/s 
     |████████████████████████████████| 120 kB 72.7 MB/s 
     |████████████████████████████████| 4.9 MB 9.9 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 6.6 MB 54.5 MB/s 
     |████████████████████████████████| 1.6 MB 6.4 MB/s 
     |████████████████████████████████| 5.3 MB 8.9 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 270 kB 74.2 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 2.3 MB 55.8 MB/s 
     |████████████████████████████████| 212 kB 67.6 MB/s 
     |████████████████████████████████| 112 kB 67.5 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |██████████████████████████

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=True).to('cuda')

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

{'trained_betas'} was not found in config. Values will be initialized to default values.


#Make images

In [ ]:
from torch import autocast
from google.colab import drive
import os
import numpy as np
import gradio as gr
import random

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def make_img(prompt, width=512, height=512, num_imgs=1, cfg_scale=7.5, steps=50):
    imgs = []
    prompt_dir = f'/content/gdrive/My Drive/AI/StableDiffusion/{prompt}'
    if not os.path.exists(prompt_dir):
        os.mkdir(prompt_dir)
    for i in range(num_imgs):
        rand= random.randint(0,10000)
        image = pipeline(prompt, width=width, height=height, scale=cfg_scale, steps=steps)["sample"][0]
        image.save(f'{prompt_dir}/({rand}).png')
        imgs.append(image)
    return imgs

In [ ]:
with gr.Blocks(title='Stable Diffusion Demo') as stable:
    with gr.Tabs() as tabs:
        with gr.TabItem('txt2img', id='txt2img_tab') as txt2img:
            with gr.Row():
                with gr.Column(scale=8):
                    prompt = gr.Textbox(label="Prompt") 
                with gr.Column(scale=1):
                    diffuse = gr.Button(value='Dream', label="Diffuse", elem_id='submit_btn')
            with gr.Row():
                height = gr.Slider(512, 1280, 512, step=64, label="Height")
                width = gr.Slider(minimum=512, maximum=1280, value=512, step=64, label="Width")
                num_imgs = gr.Slider(1, maximum=10, value=3, step=1, label="Number of Images")
                cfg_scale = gr.Slider(minimum=1, maximum=40, value=7.5, step=0.5, label="CFG_scale")
                steps = gr.Slider(minimum=1,maximum=150,value=50,step=1, label='Steps')
            gallery = gr.Gallery(label="Output", elem_id="txt2img_gallery_output").style(grid=3)
            diffuse.click(fn=make_img, inputs=[prompt, width, height, num_imgs, cfg_scale, steps], outputs=gallery)


stable.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://13678.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

...

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]